In [48]:
# Loading data 
# From 4 csv file sources 

import pandas as pd
import numpy as np
import os



# Loading customers.csv
customers = pd.read_csv("/home/sageGaurab/data-world/jupyter_stuff/csv_files/cleaned_customers.csv", parse_dates=["signup_date"])

#Loading orders.csv
orders = pd.read_csv("/home/sageGaurab/data-world/jupyter_stuff/csv_files/cleaned_orders.csv", parse_dates=["order_date"])

# Loading products.csv
products = pd.read_csv("/home/sageGaurab/data-world/jupyter_stuff/csv_files/cleaned_products.csv")

# Loading order_details.csv
order_details = pd.read_csv("/home/sageGaurab/data-world/jupyter_stuff/csv_files/cleaned_order_details.csv")


#### Joining Table 


In [49]:
orders.sample()

,order_id,customer_id,order_date,order_status
4881,6853,282,2024-08-18,Completed


In [50]:
customers.sample()

,customer_id,name,city,state,signup_date
42,43,Karen Owen,Tuckerchester,Florida,2025-05-31


In [51]:
order_details.sample()

,order_id,product_id,quantity,discount
7127,43,186,2,0.01


In [52]:
products.sample()

,product_id,product_name,category,unit_price,cost_price,cost_price_higher_than_price
100,101,Send,Food,330.99,3.18,False


In [53]:
sales = (
    orders.merge(customers, on = 'customer_id', how = 'left')
    .merge(order_details, on = 'order_id', how = 'left')
    .merge(products, on = 'product_id', how = 'left')
)

In [54]:
sales.sample()

,order_id,customer_id,order_date,order_status,name,city,state,signup_date,product_id,quantity,discount,product_name,category,unit_price,cost_price,cost_price_higher_than_price
4548,4174,1029,2025-07-27,Completed,Julie Rivas,Fritzport,Delaware,2024-03-27,19.0,8.0,0.1,Energy,Food,417.79,212.24,False


##### In Sales left most df is priotize most all the rows from orders would be included 
##### And matching customer_id will only be include from customers DataFram
##### Meaning only customers who have placed orders


### sales revenue

In [55]:
sales["revenue"] = sales["quantity"] * sales["unit_price"] # Total amount * (price/unit)

In [56]:
sales[
    ['customer_id', 'name', 'quantity', 'unit_price', 'revenue']
    ].sample(5)

,customer_id,name,quantity,unit_price,revenue
3605,1663,Joshua Anderson,2.0,446.86,893.72
3531,34,Christopher Cohen,9.0,422.69,3804.21
3371,1385,Michelle Yoder,3.0,92.37,277.11
3164,944,Susan Phillips,NaN,NaN,NaN
5247,1421,Maria Clay,NaN,NaN,NaN


### sales cost

In [57]:
sales["Total_cost"] = sales['cost_price'] * sales['quantity'] # Total cost_price of order placed

In [59]:
sales[
    ['customer_id', 'name', 'quantity', 'unit_price', 'revenue', 'Total_cost']
    ].sample(5)

,customer_id,name,quantity,unit_price,revenue,Total_cost
3053,281,Kelsey Crawford,NaN,NaN,NaN,NaN
2424,1807,Joshua Paul,3.0,382.10,1146.30,798.51
391,1410,Russell Camacho,NaN,NaN,NaN,NaN
7193,766,Steve Sanchez,6.0,244.94,1469.64,1087.50
7880,948,Amanda Sullivan,2.0,240.78,481.56,426.88


### Discounted_amount

In [75]:
sales['Discounted_amount'] = (sales['revenue'] * sales['discount'])

### profit

In [71]:
sales['Actual_revenue'] = sales['revenue']-(sales['revenue'] * sales['discount'])

### Actual revenue

In [72]:
sales["Profit"]= sales['Actual_revenue'] - sales['Total_cost'] # profit for each orders

In [78]:
sales[
    ['customer_id', 'name', 'quantity', 'unit_price', 'Total_cost', 'revenue', 'Discounted_amount', 'Actual_revenue', 'Profit']
    ].sample(5)

,customer_id,name,quantity,unit_price,Total_cost,revenue,Discounted_amount,Actual_revenue,Profit
5536,39,Julie Carrillo,1.0,330.05,126.11,330.05,99.0150,231.0350,104.9250
3018,915,Patricia Hicks,8.0,275.95,944.32,2207.60,286.9880,1920.6120,976.2920
1909,1327,Eileen Moore,2.0,87.22,69.90,174.44,15.6996,158.7404,88.8404
7290,1794,David Rowe,1.0,152.20,12.44,152.20,42.6160,109.5840,97.1440
2114,1437,Michael Levy,2.0,367.05,236.42,734.10,212.8890,521.2110,284.7910


### Groupby() 
grouping certain entites to get result by the entity
example : 
when you want to know revenue for certain product 
you group the product together and aggreate it to find the revenue 

### Revenue by month
finding revenue for each month

In [86]:
sales['month'] =  sales['order_date'].dt.to_period('M')

In [87]:
monthly_revenue = sales.groupby('month')['Actual_revenue'].sum()

In [88]:
monthly_revenue.sample(5)

month
2025-08    385063.4494
2023-07    198652.2969
2025-05    314828.6629
2025-09    300638.3630
2025-07    266638.2742
Freq: M, Name: Actual_revenue, dtype: float64

### Revenue by products

##### Ignore the product actual name they are just randomly generated strings

In [93]:
revenue_by_products = sales.groupby('product_name')['Actual_revenue'].sum()

In [94]:
revenue_by_products.sample(5)

product_name
Their        14155.6569
Energy       76752.2009
Clear        53371.1766
Political    34798.8480
Never        14024.4890
Name: Actual_revenue, dtype: float64

### Top 10 products

In [96]:
top10_products = revenue_by_products.sort_values(ascending = False).head(10)

In [97]:
print(top10_products)

product_name
Hospital       130458.6864
Example        124329.0321
Performance    120439.1153
Shake          113793.0683
Down           111480.9451
Decide         110361.4493
History        103917.4324
Fear            94239.2150
Already         92217.2786
Born            91023.9296
Name: Actual_revenue, dtype: float64


### KPI's Key Points Indicators

In [98]:
Total_revenue = sales['Actual_revenue'].sum

In [99]:
print(Total_revenue)

8638574.9856


In [100]:
Total_profit = sales['Profit'].sum()

In [101]:
print(Total_profit)

3797065.2155999998


In [103]:
AOV = sales.groupby('order_id') ['Actual_revenue'].sum().mean()

In [104]:
print(AOV)

1524.6337779032824


In [106]:
LTV = sales.groupby('customer_id') ['Actual_revenue'].sum()

In [109]:
LTV.sample(5)

customer_id
70      5061.8043
230     3242.2507
1433       0.0000
698     9460.0590
1026    1013.6520
Name: Actual_revenue, dtype: float64